In [1]:
import pandas as pd

This Jupyter Notebook serves to prepare ProteoBench inputs from software tool outputs that have important information, needed by ProteoBench, in more than one file

# MSAID output preparation

In [ ]:
# Change paths to your local paths
input_path = "./path/to/your/MSAID/output/dir/"  # path should contain the proteingroups.tsv and precursors.tsv files
output_path = "./path/to/your/desired/output/dir/"  # path where the output files will be saved

In [ ]:
# Change the path to your proteingroups.tsv and precursors.tsv files
protein_file = pd.read_csv(input_path + "proteingroups.tsv", sep="\t")
precursor_file = pd.read_csv(input_path + "precursors.tsv", sep="\t")

In [ ]:
# Map the proteins to the precursors using the "PROTEIN_IDS" column in the precursor file
def add_fasta_headers(prec_df, protein_df):
    # Create a dictionary from the second DataFrame for fast look-up
    protein_to_header = dict(zip(protein_df["PROTEIN_IDS"], protein_df["FASTA_HEADERS"]))

    # Function to find and join headers for each PROTEIN_IDS entry
    def get_fasta_headers(protein_ids):
        ids = protein_ids.split(";")  # Split the IDs by the separator
        headers = [protein_to_header.get(protein_id.strip(), "") for protein_id in ids]
        headers = [header for header in headers if header]  # Remove empty headers
        return "; ".join(headers) if headers else None

    # Apply the function to the PROTEIN_IDS column and create a new FASTA_HEADERS column
    prec_df["FASTA_HEADERS"] = prec_df["PROTEIN_IDS"].apply(get_fasta_headers)

    return prec_df

In [20]:
prec_df_with_headers = add_fasta_headers(precursor_file, protein_file)

In [ ]:
# Change the path to the output file
prec_df_with_headers.to_csv(
    output_path + "precursors_with_headers.tsv", sep="\t", index=False
)  # This file can be uploaded to ProteoBench

# AlphaDIA output preparation

In [ ]:
input_path = (
    "/path/to/your/AlphaDIA/output/dir"  # path should contain the precursors.tsv file and the precursor.matrix.tsv file
)
output_path = "/path/to/your/desired/output/path"  # path where the output files will be saved

In [3]:
precursors_long = pd.read_csv(input_path + "precursors.tsv", sep="\t", dtype={"mod_seq_charge_hash": str})
precursor_matrix = pd.read_csv(input_path + "precursor.matrix.tsv", sep="\t", dtype={"mod_seq_charge_hash": str})

In [8]:
precursor_matrix_with_precursor_info = pd.merge(
    precursor_matrix,
    precursors_long[["genes", "decoy", "mods", "mod_sites", "sequence", "charge", "mod_seq_charge_hash"]],
    on="mod_seq_charge_hash",
)
precursor_matrix_with_precursor_info.drop_duplicates(inplace=True)

In [9]:
precursor_matrix_with_precursor_info.to_csv(
    output_path + "precursor_matrix_with_precursor_info_fixed.tsv", sep="\t", index=False
)  # This file can be uploaded to ProteoBench